In [47]:
corpus = sc.textFile("WikipediaPagesOneDocPerLine1000LinesSmall.txt")
# fucntion to count number of occurences of eacch word in each documents and 
# calculate term frequences
def buildArray (listOfIndices):
    returnVal = np.zeros (20000)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    mysum = np.sum (returnVal)
    returnVal = np.divide (returnVal, mysum)
    return returnVal

# function to get array whose element is 0 if the tf is 0, otherwise 1
def zeroOneArray(listOfTFs):
    returnVal = np.zeros(20000)
    for i in range(len(listOfTFs)):
        if listOfTFs[i] != 0.0:
            returnVal[i] = 1.0
    return returnVal

import sys
from operator import add
import re

import numpy as np

from operator import add
# create number of docs
numberOfDocs = corpus.count()

# filter out lines where do not have id and url field
validLines = corpus.filter(lambda x : 'id' in x and 'url=' in x)

# map to key value pairs whose key is id and value is document body..
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))

# create regex object, make document all lowercase and split by " "
# [('431949', ['black', 'people', 'and', 'mormonismwhile'...
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# flat map each word to key value pair like ("word", 1)
# ex [('black', 1), ('people', 1), ('and', 1), ('mormonismwhile', 1), ('at', 1)......
allWords =keyAndListOfWords.flatMap(lambda x: x[1]).map(lambda x: (x, 1))

# aggregate to cout
# [('at', 4810), ('two', 1848), ('priesthood', 78), ('in', 27758), ('mid', 159).....
allCounts = allWords.reduceByKey(add)

# get top 20000 words
topWords = allCounts.top(20000, key=lambda x: x[1])

# get key value pairs whose key is word and value is position rank of within top 20000
twentyK = sc.parallelize(range(20000))
dictionary = twentyK.map (lambda x : (topWords[x][0], x))

# get key value pairs whose keys are all words in documnt and values are ids
# (word, id)
allWords = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

# join rdd to get key value pairs like ("word", (rank, id))
allDictionaryWords = dictionary.join(allWords)

# map to (id, rank)
justDocAndPos = allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))

# group by id 
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

# aggregate to numpy array of ranks and concatinate string 
forCSV= allDictionaryWordsInEachDoc.mapValues(lambda x: np.array( list(x)))



In [48]:
###### From here task2

# calculate term frequences for each word in each document
tfs = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))

In [49]:
tfs.take(5)

[('431971',
  array([0.08553655, 0.02488336, 0.05132193, ..., 0.        , 0.        ,
         0.        ])),
 ('431999',
  array([0.06479482, 0.03239741, 0.03887689, ..., 0.        , 0.        ,
         0.        ])),
 ('432000',
  array([0.07192043, 0.04475899, 0.02371844, ..., 0.        , 0.        ,
         0.        ])),
 ('432001',
  array([0.08540752, 0.03318692, 0.02781845, ..., 0.        , 0.        ,
         0.        ])),
 ('432035',
  array([0.07300672, 0.04130644, 0.05091258, ..., 0.        , 0.        ,
         0.        ]))]

In [57]:
# calculate array whose elememts are 0 or 1
zeroOrOne = tfs.map(lambda x: (x[0], zeroOneArray(x[1])))

In [56]:
import sys
zeroOrOne.take(5)
np.set_printoptions(threshold=sys.maxsize)

In [58]:
zeroOrOne.take(5)

[('431971',
  array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
         1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [59]:
dfs = zeroOrOne.reduce (lambda x1, x2:(("", np.add (x1[1], x2[1]))))[1]

In [61]:
dfs

array([919., 920., 869., 889., 922., 884., 750., 775., 792., 752., 798.,
       715., 785., 767., 688., 425., 742., 461., 668., 662., 703., 547.,
       664., 536., 583., 552., 599., 499., 639., 527., 600., 539., 512.,
       485., 556., 557., 449., 500., 539., 457., 529., 536., 450., 463.,
       508., 450., 491., 438., 481., 427., 410., 349., 427., 326., 427.,
       477., 399., 188., 426., 389., 212., 383., 309., 395., 407., 330.,
       390., 372., 389., 346., 369., 233., 327., 276., 353., 375., 261.,
       385., 237., 341., 367., 317., 290., 333., 258., 397., 343., 232.,
       367., 349.,  86., 243., 352., 387., 370., 364., 336., 354., 248.,
       384., 192., 335., 229., 240., 319., 242., 273., 338., 243., 217.,
       181., 323., 132., 168., 345., 327., 271., 287., 315., 285., 200.,
       178., 163., 235., 271., 259., 182.,  96., 180., 285., 214., 115.,
       299., 298., 281., 274., 244., 229., 187., 148.,  97., 256., 164.,
        86., 236., 243., 147., 191., 139., 285., 18

In [62]:
multiplier = np.full (20000, numberOfDocs)
idfs = np.log (np.divide(numberOfDocs, dfs))

In [63]:
idfs

array([0.08446916, 0.08338161, 0.14041215, 0.11765804, 0.08121006,
       0.12329822, 0.28768207, 0.25489225, 0.23319389, 0.28501896,
       0.22564668, 0.33547274, 0.24207156, 0.26526848, 0.37396644,
       0.85566611, 0.29840604, 0.77435724, 0.40346711, 0.41248972,
       0.35239839, 0.60330648, 0.40947313, 0.62362112, 0.53956809,
       0.59420723, 0.51249368, 0.69514918, 0.44785082, 0.64055473,
       0.51082562, 0.61803971, 0.66943065, 0.72360639, 0.58698698,
       0.58519004, 0.80073239, 0.69314718, 0.61803971, 0.78307189,
       0.63676685, 0.62362112, 0.7985077 , 0.77002822, 0.67727383,
       0.7985077 , 0.71131115, 0.82553637, 0.73188801, 0.85097127,
       0.89159812, 1.05268336, 0.85097127, 1.1208579 , 0.85097127,
       0.74023879, 0.91879386, 1.67131332, 0.85331593, 0.94417594,
       1.551169  , 0.95972029, 1.174414  , 0.92886951, 0.89894209,
       1.10866262, 0.94160854, 0.98886142, 0.94417594, 1.0613165 ,
       0.99695863, 1.45671683, 1.11779511, 1.28735441, 1.04128

In [64]:
###### From here task2

# calculate term frequences for each word in each document
tfs = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))

# calculate array whose elememts are 0 or 1
zeroOrOne = tfs.map(lambda x: (x[0], zeroOneArray(x[1])))

# calculate word in document occurences within the corpuse
dfs = zeroOrOne.reduce (lambda x1, x2:(("", np.add (x1[1], x2[1]))))[1]

# calculate inverse document frequency 
multiplier = np.full (20000, numberOfDocs)
idfs = np.log (np.divide(numberOfDocs, dfs))

#  calculate tfidf
tfidfs = tfs.map (lambda x: (x[0], np.multiply (x[1], idfs)))

In [23]:
# textInput = 'Sport Basketball Volleyball Soccer'
# target = sc.parallelize(("", textInput))
# wordsArray = target.flatMap(lambda x: ((i, 1) for i in regex.sub(' ', x).split()))
# dictionaryWords = dictionary.join(wordsArray)
# allDictionaryWordsInThatDoc = dictionaryWords.map(lambda x: (x[1][1], x[1][0]))\
#                                             .groupByKey()\
#                                             .mapValues(lambda x: np.array(list(x)))

In [24]:
# textInput = 'How many goals Vancouver score last year?'
# target = sc.parallelize(("", textInput))
# wordsArray = target.flatMap(lambda x: ((i, 1) for i in regex.sub(' ', x).split()))
# dictionaryWords = dictionary.join(wordsArray)
# allDictionaryWordsInThatDoc = dictionaryWords.map(lambda x: (x[1][1], x[1][0]))\
#                                             .groupByKey()\
#                                             .mapValues(lambda x: np.array(list(x)))
# targetTfs = buildArray(allDictionaryWordsInThatDoc.top(1)[0][1])
# targetZeroOrOne = zeroOneArray(targetTfs) 
# targetDf = np.sum(targetZeroOrOne)
# targetIdfs = np.log(numberOfDocs / df)
# targetTfidfs = np.multiply(targetTfs, targetIdfs)
# distances = tfidfs.map(lambda x: (x[0], np.dot(x[1], targetTfidfs)))
# topK = distances.top(30, lambda x: x[1])
# docIDRepresented = sc.parallelize(topK).map (lambda x : (x[0], 1))
# numTimes = docIDRepresented.reduceByKey(lambda x: add)

In [65]:
def getPrediction(textInput, k):
        
        # make rdd for document 
        target = sc.parallelize(("", textInput))
        # (word, 1)
        wordsArray = target.flatMap(lambda x: ((i, 1) for i in regex.sub(' ', x).lower().split()))
        # dictionary(word, dicPosition) join with wordArray(word, 1) to make (word ,(dicPosition, 1))
        dictionaryWords = dictionary.join(wordsArray)
        # groupby to count number of each word in the new document
        # (1, [dicPosi, dicPosi])      
        allDictionaryWordsInThatDoc = dictionaryWords.map(lambda x: (x[1][1], x[1][0]))\
                                                .groupByKey()
                                                #.mapValues(lambda x: np.array(list(x)))
        # calculate term frequences for each word in each document
        targetTfs = buildArray(allDictionaryWordsInThatDoc.top(1)[0][1])
        
        # calculate array whose elememts are 0 or 1
        # targetZeroOrOne = zeroOneArray(targetTfs)
        # calculate document occurences within the corpuse
        # targetDf = np.sum(targetZeroOrOne)

        # targetIdfs = np.log(numberOfDocs / targetDf)

        targetTfidfs = np.multiply(targetTfs, idfs)
        # calculate cosine similality between textnput and all docs
        distances = tfidfs.map(lambda x: (x[0], np.dot(x[1], targetTfidfs)))
        # select top k similarity (docID, similaliry)
        topK = distances.top(k, lambda x: x[1])
        # count how many time each docID appear in the top k similar docID
        docIDRepresented = sc.parallelize(topK).map (lambda x : (x[0], 1))
        numTimes = docIDRepresented.reduceByKey(add)
        # take top 1
        return numTimes.top(1, lambda x: x[1])

In [66]:
print(getPrediction('Sport Basketball Volleyball Soccer', 30))

[('432151', 1)]


In [67]:
print(getPrediction("How many goals Vancouver score last year?", 30))

[('454889', 1)]
